In [1]:
import sys
sys.path.append('/gale/netapp/home/wtian/local/pyballc/pyballc/')
import glob


In [2]:
import pyballcools
import pysam

ModuleNotFoundError: No module named 'pyballcools'

In [ ]:
ballc_file = '/gale/netapp/home/wtian/local/ballcools/test/out/HBA_200616_H1930001_CX50_M1C_1_P5-3-E10-A17.ballc'
cmeta_file = '/gale/netapp/home/wtian/bican/projects/test/h1930001.cmeta.gz'

In [ ]:
next(pysam.TabixFile(cmeta_file).fetch(*region))

In [ ]:
region = 'chr1', 0,20000
# region = 'chr1', 62000, 70000
# region = 'chr1', 0,10000
region = 'chr1', 0, 80000


In [ ]:
class BAllCFile:
    def __init__(self, ballc_file, cmeta_file=None):
        self.bci = pyballcools.BAllCIndex(ballc_file)
        self.tbi = pysam.TabixFile(cmeta_file) if cmeta_file is not None else None
    def _fetch_with_cmeta(self, chrom, start, end):
        mciter = self.bci.QueryMcRecords_Iter(chrom, start, end)
        if mciter.HasNext():
            mciter.Next()
        while mciter.HasNext():
            rec = mciter.Next()
            try:
                cline = next(self.tbi.fetch(rec.chrom,rec.pos-1,rec.pos,))
            except:
                ##TODO
                pass

            *_, strand, context = cline.split()

            yield(rec.chrom,rec.pos,strand, context, rec.mc,rec.cov, )
    def _fetch(self, chrom, start, end):
        mciter = self.bci.QueryMcRecords_Iter(chrom, start, end)
        if mciter.HasNext():
            mciter.Next()
        while mciter.HasNext():
            rec = mciter.Next()
            yield(rec.chrom, rec.pos, rec.mc, rec.cov, )
        
    def fetch(self, chrom, start, end):
        if self.tbi is None:
            return self._fetch(chrom, start, end)
        else:
            return self._fetch_with_cmeta(chrom, start, end)

In [ ]:
ballc = BAllCFile(ballc_file, cmeta_file)

In [ ]:
for x in ballc.fetch(*region):
    print(x)